In [1]:
from pathlib import Path
import subprocess

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display
from PIL import Image

In [2]:
# Pillow didn't work to convert 16bit *.jp2 into PDFs so . . . 
# We'll try converting them to JPG first with ImageMagick then maybe pipe
# that output as input for Pillow to make a PDF as it's quick

In [3]:
directory_of_directories_to_convert_to_PDF_path = Path('/Users/dlisla/Pictures/athletics-programs/Output/toJpeg')

convert_to_PDF_directory_paths_list = sorted([x for x in directory_of_directories_to_convert_to_PDF_path.iterdir() if x.is_dir()])

number_of_directories = len(convert_to_PDF_directory_paths_list)
print(f'{number_of_directories} directories to process into PDFs')

5 directories to process into PDFs


In [4]:
# batch process

# progress bar
progress_label = Label('Directories to convert to PDF')
progress_bar = IntProgress(min=0, max=number_of_directories)
progress_widget = VBox([progress_label, progress_bar])
display(progress_widget)


for index, directory_path in enumerate(convert_to_PDF_directory_paths_list, start=1):
    
    label = f'Processing {directory_path.name} . . . '
    progress_label.value = label
    
    images_list = []
    print(f'Starting find convert in {directory_path}')
    # find and convert everything with imagemagick to jpeg
    !find {str(directory_path)} -name "*.jp2" -print0 | xargs -0 -I [] -n 1 -P 16 magick "[]" -strip -resize 2500x2500 -quality 60 -density 72x72 -units pixelsperinch "[]".jpg
    progress_bar.value = index

# run the validation as a 2nd pass so there aren't race issues
for index, directory_path in enumerate(convert_to_PDF_directory_paths_list, start=1):
    
    label = f'Verifying {directory_path.name} . . . '
    progress_label.value = label
    error_list = []
    images_list = []
    print(f'Starting to validate convert in {directory_path}')
    image_paths_list = sorted(directory_path.glob('*.jp2'))
    for image_path in image_paths_list:
        jp2_jpg_name = f'{image_path.name}.jpg'
        jp2_jpg_path = directory_path.joinpath(jp2_jpg_name)
        # remove *.jp2.jpg because it bugs me
        jpg_name = f'{image_path.stem}.jpg'
        jpg_path = directory_path.joinpath(jpg_name)
        jp2_jpg_path.rename(jpg_path)
        
        # test if jpg_path is a file or not
        if jpg_path.is_file():  # delete *.jp2
            image_path.unlink()
        else:  # add the image to our error list
            error_list.append(jpg_path)
    if error_list:
        print('*************************************************** ERRORS:')
        print(f'{error_list}\n')
    else:
        print(f'{directory_path.name} has all jpgs\n')
    progress_bar.value = index

Starting find convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1987-05
Starting find convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1988
Starting find convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1989
Starting find convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1990
Starting find convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1991
Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1987-05
vols-track-and-field_1987-05 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1988
vols-track-and-field_1988 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/toJpeg/vols-track-and-field_1989
vols-track-and-field_1989 has all 

In [46]:
# run the validation as a 2nd pass so there aren't race issues
for index, directory_path in enumerate(convert_to_PDF_directory_paths_list, start=1):
    
    label = f'Verifying {directory_path.name} . . . '
    progress_label.value = label
    error_list = []
    print(f'Starting to validate convert in {directory_path}')
    image_paths_list = sorted(directory_path.glob('*.jp2'))
    for image_path in image_paths_list:
        jpg_name = f'{image_path.name}.jpg'
        jpg_path = directory_path.joinpath(jpg_name)
        if not jpg_path.is_file():
            error_list.append(jpg_path)
    if error_list:
        print('*************************************************** ERRORS:')
        print(f'{error_list}\n')
    else:
        print(f'{directory_path.name} has all jpgs\n')
    progress_bar.value = index

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/vols-basketball_batch2/vols-basketball_1988
vols-basketball_1988 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/vols-basketball_batch2/vols-basketball_1989
vols-basketball_1989 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/vols-basketball_batch2/vols-basketball_1990
vols-basketball_1990 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/vols-basketball_batch2/vols-basketball_1991
vols-basketball_1991 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/vols-basketball_batch2/vols-basketball_1992
vols-basketball_1992 has all jpgs

Starting to validate convert in /Users/dlisla/Pictures/athletics-programs/Output/vols-basketball_batch2/vols-basketball_1993
vols-basketball_1993 has all jpgs

Starting to validate convert in /Users/d

In [45]:
s = 'find . -name "*.jp2" -print0 | xargs -0 -I {} -n 1 -P 16 magick "{}" -strip -resize 4000x4000 -quality 60 -density 72x72 -units pixelsperinch "{}".jpg'
s_list = s.split(' ')
print(s_list)

['find', '.', '-name', '"*.jp2"', '-print0', '|', 'xargs', '-0', '-I', '{}', '-n', '1', '-P', '16', 'magick', '"{}"', '-strip', '-resize', '4000x4000', '-quality', '60', '-density', '72x72', '-units', 'pixelsperinch', '"{}".jpg']


In [32]:
test_path = Path('/Users/dlisla/Pictures/athletics-programs/Output/toJhove/vols-baseball_1986/vols-baseball_1986_001.jpg')
test_path.exists()

False